In [ ]:
import ee
import geemap

geemap.ee_initialize()

# Exporting images

In [ ]:
Map = geemap.Map()

image = ee.Image("LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318").select(
    ["B5", "B4", "B3"]
)

vis_params = {"min": 0, "max": 0.5, "gamma": [0.95, 1.1, 1]}

Map.centerObject(image, 8)
Map.addLayer(image, vis_params, "Landsat")
Map

In [ ]:
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
fc = ee.FeatureCollection(region)
style = {"color": "ffff00ff", "fillColor": "00000000"}
Map.addLayer(fc.style(**style), {}, "ROI")
Map

# To local Drive

not allow to direct download the whole Landsat images (file size too big)

In [ ]:
geemap.ee_export_image(image, filename="landsat.tif", scale=30, region=region)

# Projection

In [ ]:
projection = image.select(0).projection().getInfo()
projection

In [ ]:
crs = projection["crs"]
crs_transform = projection["transform"]

In [ ]:
geemap.ee_export_image(
    image,
    filename="landsat_crs.tif",
    crs=crs,
    crs_transform=crs_transform,
    region=region,
)

## No restrcition way (download the large data file)

`geemap.download_ee_image`

slice the images, and download it piece by piece

In [ ]:
geemap.download_ee_image(image, filename="landsat_full.tif", scale=60)

## fishnet method

In [ ]:
fishnet = geemap.fishnet(image.geometry(), rows=4, cols=4, delta=0.5)
style = {"color": "ffff00ff", "fillColor": "00000000"}
Map.addLayer(fishnet.style(**style), {}, "Fishnet")
Map

In [ ]:
outdir = "tiles"
geemap.download_ee_image_tiles(
    image, fishnet, outdir, prefix="landsat_", crs="EPSG:3857", scale=30
)

# Similar and faster way
`geemap.download_ee_image_tiles_parallel`

In [ ]:
out_dir = "tiles2"
geemap.download_ee_image_tiles_parallel(
    image,
    fishnet,
    out_dir,
    prefix="landsat_",
    crs="EPSG:3857",
    scale=120,
)

# To Google Drive

Run on space, store the data in google drive

Run on backspace

pros: work for large datasets

cons: need to wait

In [ ]:
# need change authenticate settings
geemap.ee_export_image_to_drive(
    image, description="landsat_image", folder="export", region=region, scale=30
)

# To mumpy array

small dataset

In [ ]:
region = ee.Geometry.BBox(-122.5003, 37.7233, -122.3410, 37.8026)
rgb_img = geemap.ee_to_numpy(image, region=region)

In [ ]:
print(rgb_img.shape)

In [ ]:
import matplotlib.pyplot as plt

rgb_img_test = (255 * ((rgb_img[:, :, 0:3]) + 0.2)).astype("uint8")
plt.imshow(rgb_img_test)
plt.show()